In [ ]:
#https://www.kaggle.com/krooz0/resnet-features-and-svr/notebook

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
import os

In [ ]:
train_df=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_df.head()

In [ ]:
import tensorflow
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
def create_model():
    conv_base=EfficientNetB4(weights='../input/tf-keras-pretrained-model-weights/Top/efficientnetb4.h5')
    conv_base=Model(inputs=conv_base.input, outputs=conv_base.get_layer("avg_pool").output)#(None, 1792)

    top_model = models.Sequential()
    top_model.add(layers.Dense(512, activation="relu"))
    top_model.add(layers.Dense(32, activation="relu"))
    
    model = Model(inputs=conv_base.input, outputs=top_model(conv_base.output))

    return model


cnn_model=create_model()

In [ ]:
from tqdm import tqdm

In [ ]:
image_size=380 #EfficientNet

features_array=np.zeros((9912,32))#len(train_df)=3312


for i, file in enumerate(tqdm(train_df["Id"])):
    img = image.load_img(os.path.join('../input/petfinder-pawpularity-score/train',file+'.jpg'), target_size=(image_size, image_size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = cnn_model.predict(x)
    features=features.reshape(32,)
    features_array[i,:]=features

In [ ]:
#import pickle
#with open("features_array.pkl", "wb") as f:
    #pickle.dump(features_array, f)

In [ ]:
print(features_array.shape)

train_feature_df= pd.DataFrame(features_array)
train_feature_df = pd.concat([train_df,train_feature_df], axis=1, join='inner')
train_feature_df.head()

In [ ]:
"""from sklearn.model_selection import train_test_split
train_set,val_set = train_test_split(train_feature_df,test_size=0.2,random_state=42)

X_train=train_set.drop(['Id','Pawpularity'],axis=1)
y_train=train_set['Pawpularity']

X_val=val_set.drop(['Id','Pawpularity'],axis=1)
y_val=val_set['Pawpularity']"""

In [ ]:
X=train_feature_df.drop(["Id", "Pawpularity"], axis=1)
y=train_feature_df["Pawpularity"]

In [ ]:
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm

In [ ]:
"""#パラメータ調節
def param():
    param_grid= {
        'metric':['rmse'],
        'n_estimators':[5,10,25,50,100],
        'num_leaves': [5,10,15,30],
        "max_depth":[5,10,50,100],
        'learning_rate': [0.01,0.05,0.1]
    }
    
    return param_grid


model=lgbm.LGBMRegressor(
    random_state=42,
    verbose=0
)


grid_search = GridSearchCV(
    model,
    param(),
    cv=KFold(n_splits=5),
    n_jobs=-1
)"""

In [ ]:
#grid_search.fit(X,y)

In [ ]:
#print(grid_search.best_estimator_)
#print(grid_search.best_score_)

In [ ]:
best_model=lgbm.LGBMRegressor(learning_rate=0.05,max_depth=5,num_leaves=15,metrics="rmse",random_state=42,verbose=0)

In [ ]:
#import pickle
#with open("best_model.pkl", "wb") as f:
    #pickle.dump(best_model, f)

In [ ]:
#LightGBM prediction
best_model.fit(X,y)

In [ ]:
test_df=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df.head()

In [ ]:
image_size=380 #EfficientNet

test_features_array=np.zeros((len(test_df),32))#len(test_df)=8


for i, file in enumerate(tqdm(test_df["Id"])):
    img = image.load_img(os.path.join('../input/petfinder-pawpularity-score/test',file+'.jpg'), target_size=(image_size, image_size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_features = cnn_model.predict(x)
    test_features=features.reshape(32,)
    test_features_array[i,:]=features

In [ ]:
test_feature_df= pd.DataFrame(test_features_array)
test_feature_df = pd.concat([test_df,test_feature_df], axis=1, join='inner')
test_feature_df.head()

In [ ]:
test_y_pred=best_model.predict(test_feature_df.drop(columns=["Id"]))

In [ ]:
submission=pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

In [ ]:
submission["Id"]=test_df["Id"]
submission["Pawpularity"]=test_y_pred

submission.head(5)

In [ ]:
submission.to_csv("submission.csv",index=False)